In [1]:
from fenics import *
import h5py 
import numpy as np
import pickle


In [2]:
def read_h5_fenics(filename, dataset="concentration_0"):
    """
    Function used to read nodal values from H5 files.

    Parameters
    ----------
    filename : str
        String containing the filename for reading files in h5 (libMesh and EdgeCFD).
    dataset : str
        String containing the dataset desired.

    Returns
    -------
    array : np.array
        Numpy array containing nodal values.
    """
    h5_file = h5py.File(filename, "r")
    for key in h5_file.keys():
        group = h5_file[key]
        for key in group.keys():
            data = group[(dataset)]
    data_array = np.array(data["vector"], copy=True)
    h5_file.close()
    return data_array

def write_h5_fenics(filename, data_array dataset="concentration_0"):
    """
    Function used to read nodal values from H5 files.

    Parameters
    ----------
    filename : str
        String containing the filename for reading files in h5 (libMesh and EdgeCFD).
    dataset : str
        String containing the dataset desired.

    Returns
    -------
    array : np.array
        Numpy array containing nodal values.
    """
    h5_file = h5py.File(filename, "r+")
    h5_file["concentration"]["concentration_0"]["vector"] = data_array
    h5_file.close()
    return data_array

In [17]:
def insert_data_to_fenics(snapshot, t, out_c):
    solution = Function(W)
    solution.rename("concentration", "c/c0")
    solution.vector()[:] = np.squeeze(snapshot)
    out_c << (solution, t)
    #out_c.write_checkpoint(solution, "conc", t)
    return 


In [23]:
FILE_C = f"reconstruction_results_serial/concentration_reconstructed.pvd" # NOME DO ARQUIVO DE SAÍDA
#out_c = XDMFFile(FILE_C)
out_c = File(FILE_C, "compressed")

READ_PICKLE = False 
HEIGHT = 2.05
WIDTH = 18.0
NX = 700
NY = 100

comm = MPI.comm_world
rank = comm.Get_rank()

parameters["form_compiler"]["optimize"] = True
parameters["form_compiler"]["cpp_optimize"] = True


mesh = RectangleMesh(Point(0.0, 0.0), Point(WIDTH, HEIGHT), NX, NY)

P1 = FiniteElement("Lagrange", mesh.ufl_cell(), 1)
W = FunctionSpace(mesh, P1)

if READ_PICKLE:
    with open("pod_270_DL_16.pkl", "rb") as f:
        x = pickle.load(f)
else:
    first_snapshot = read_h5_fenics("results_serial/concentration_1.h5")
    rows = first_snapshot.shape[0]
    columns = 101
    x = np.zeros((rows, columns))
    x[:, 0] = np.squeeze(first_snapshot)
    for i in range(1, columns):
        data = read_h5_fenics(f"results_serial/concentration_{i}.h5")
        x[:, i] = np.squeeze(data)
 

for i in range(x.shape[1]): # LOOP POR TODOS OS VETORES APROXIMADOS PELO ROM
    reconstruction = x[:, i] 
    insert_data_to_fenics(reconstruction, i, out_c) # 

In [24]:
filename = "h5_changed_results/concentration_1.h5"
h5_file = h5py.File(filename, "r+")

In [25]:
h5_file

<HDF5 file "concentration_1.h5" (mode r+)>

In [26]:
h5_file.keys()

<KeysViewHDF5 ['concentration']>

In [31]:
h5_file["concentration"]["concentration_0"]["vector"]

<HDF5 dataset "vector": shape (70801, 1), type "<f8">